<a href="https://colab.research.google.com/github/ecaldwe1/DSCI502_DataMiningAtScale/blob/main/Week4_PageRankNotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
M = [[0, 0.5, 0, 0],
	  [1/3, 0, 1, 0.5],
	  [1/3, 0, 0, 0.5],
	  [1/3, 0.5, 0, 0]]
# fix errors inputing M
M[1][2]=0
M[0][2]=1
M # verify M is correct

[[0, 0.5, 1, 0],
 [0.3333333333333333, 0, 0, 0.5],
 [0.3333333333333333, 0, 0, 0.5],
 [0.3333333333333333, 0.5, 0, 0]]

In [ ]:
v = .25 * np.ones(4)
v

array([0.25, 0.25, 0.25, 0.25])

In [ ]:
v=np.dot(M,v)
v #[0.375, 0.208333, 0.208333, 0.208333

array([0.375     , 0.20833333, 0.20833333, 0.20833333])

In [ ]:
v=np.dot(M,v)
v

array([0.3125    , 0.22916667, 0.22916667, 0.22916667])

In [ ]:
v=np.dot(M,v)
v

array([0.328125  , 0.22395833, 0.22395833, 0.22395833])

In [ ]:
# if we do this many times, we see that v starts to converge
for i in range(10):
  v=np.dot(M,v)
  print(v)

[0.33333588 0.22222137 0.22222137 0.22222137]
[0.33333206 0.22222265 0.22222265 0.22222265]
[0.33333397 0.22222201 0.22222201 0.22222201]
[0.33333302 0.22222233 0.22222233 0.22222233]
[0.33333349 0.22222217 0.22222217 0.22222217]
[0.33333325 0.22222225 0.22222225 0.22222225]
[0.33333337 0.22222221 0.22222221 0.22222221]
[0.33333331 0.22222223 0.22222223 0.22222223]
[0.33333334 0.22222222 0.22222222 0.22222222]
[0.33333333 0.22222222 0.22222222 0.22222222]


In [ ]:
# one more update
v=np.dot(M,v)
v

array([0.33333334, 0.22222222, 0.22222222, 0.22222222])

The stochastic matrix that we have (columns summing to 1), guaranteed to be one vector $v$, for which there is an eigenvalue of 1 ($v$ would be the eigenvector). 

Our iterate $v_n$ will converge to vector $v$, where the sum of the $v_i$ is 1 as well (probability vector)


The simplified page rank of some page $j$, will be the $j^{th}$ coordinate (value of $v(j)$) where $v$ is the steady-state vector described above.

In [ ]:
for i in range(100):
  v=np.dot(M,v)

v

array([0.33333333, 0.22222222, 0.22222222, 0.22222222])

In [ ]:
v=np.dot(M,v)
v

array([0.33333333, 0.22222222, 0.22222222, 0.22222222])

we can see that v converges to [1/3, 2/9, 2/9, 2/9]


# Week 6
## Page Rank (continued)

### The Problems with our Assumptions
The assumption that the graph is strongly-connected and contains no dead-ends does not reflect the reality of the true web-graph.

Now we will take a look at how the algorithm is impacted by the presence of a dead-end.

In [ ]:
# dead-end example
M = np.array([[0,0.5,0,0],[1/3,0,0,0.5], [1/3,0,0,0.5], [1/3,0.5,0,0]])
M

array([[0.        , 0.5       , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

In [ ]:
v=0.25*np.ones(4)
v

array([0.5, 0.5, 0.5, 0.5])

In [ ]:
for i in range(100):
  v=np.dot(M,v)

In [ ]:
v

array([6.85655349e-15, 9.99292692e-15, 9.99292692e-15, 9.99292692e-15])

It is possible that the graph contains no dead-ends, but is not strongly connected. This leads to problems which are known as *spider traps*.

In [ ]:
# not strongly connected example
M2 = np.array([[0,0.5,0,0],[1/3,0,0,0.5], [1/3,0,1,0.5], [1/3,0.5,0,0]])
M2

array([[0.        , 0.5       , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.        , 1.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

In [ ]:
v2 = 0.25*np.ones(4)
v2

array([0.25, 0.25, 0.25, 0.25])

In [ ]:
for i in range(100):
  v2=np.dot(M2,v2)

v2

array([3.42827674e-15, 4.99646346e-15, 1.00000000e+00, 4.99646346e-15])

We see that all the probability is concentrated at C.
The naive simplified pagerank would suggest that C is a very important web page, which may not be the case. 

But it makes sense since any other vertex will eventually reach C and have nowhere to go from there.

As we have seen the presence of dead-ends and spider traps can render the PageRank algorithm ineffective. A means of escaping from a dead-end or spider trap is desirable.

To this end, we will introduce the concept of teleportation.
Teleporation refers to a process by which the random-surfer may choose not to follow a link on the current page and teleport immediately to a randomly selected page contained in the web.

The decision to teleport is determined randomly by a parameter beta. In other words, let beta give the probability that the surfer follows a randomly chosen link. The iteration can be summarized as

$v_n = (1-β)v_0+βMv_{n-1}$

It can be shown that the iterates will converge to a non-zero vector $v$. This vector will give the PageRanks.

In [ ]:
# Teleportation example
M3 = M2
M3

array([[0.        , 0.5       , 0.        , 0.        ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.        , 1.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

In [ ]:
v3=0.25*np.ones(4)
v3

array([0.25, 0.25, 0.25, 0.25])

In [ ]:
v0=0.25*np.ones(4)
v0

array([0.25, 0.25, 0.25, 0.25])

In [ ]:
b=0.85

In [ ]:
v3=(1-b)*v0+b*np.dot(M3,v3)
v3

array([0.14375   , 0.21458333, 0.42708333, 0.21458333])

In [ ]:
for i in range(100):
  v3=(1-b)*v0+b*np.dot(M3,v3)

v3

array([0.08249313, 0.10586618, 0.70577452, 0.10586618])

We see now, that while C has a higher probability, we do have non-zero probabilities for the other nodes. 

At this point, we may want to remove dead-ends and then apply this method. Other methods exist that would deal with larger webs, but you can read about them in the text Mining of Massive Data Sets.